In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup, Comment
import requests
import time

Below I am reading in CSV files that I downloaded from profootballfocus.com for the years and positions that I am looking at, from 2018-2009.  Each CSV represented one season of play for one position, so you can see below that I am joining all the years together for each position, creating dataframes for each differnet category (passing, rushing, receiving, defense), and creating a unique key for each player and the year they are playing in that I will use later to map this data to my larger dataset.

In [3]:
draft_years = ['2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009']
passing_df = pd.DataFrame()

for year in draft_years:
    temp_df = pd.read_csv('passing_summary_' + year + '.csv')
    temp_df['key'] = temp_df['player'] + year
    temp_df['year'] = int(year)
    passing_df = pd.concat([passing_df, temp_df])

In [ ]:
draft_years = ['2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009']
rushing_df = pd.DataFrame()

for year in draft_years:
    temp_df = pd.read_csv('./data/targets/rushing/rushing_summary_' + year + '.csv')
    temp_df['key'] = temp_df['player'] + year
    temp_df['year'] = int(year)
    rushing_df = pd.concat([rushing_df, temp_df])

In [ ]:
draft_years = ['2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009']
receiving_df = pd.DataFrame()

for year in draft_years:
    temp_df = pd.read_csv('./data/targets/receiving/receiving_summary_' + year + '.csv')
    temp_df['key'] = temp_df['player'] + year
    temp_df['year'] = int(year)
    receiving_df = pd.concat([receiving_df, temp_df])

In [ ]:
draft_years = ['2018', '2017', '2016', '2015', '2014', '2013', '2012', '2011', '2010', '2009']
defense_df = pd.DataFrame()

for year in draft_years:
    temp_df = pd.read_csv('./data/targets/defense/defense_summary_' + year + '.csv')
    temp_df['key'] = temp_df['player'] + year
    temp_df['year'] = int(year)
    defense_df = pd.concat([defense_df, temp_df])

In [4]:
passing_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 904 entries, 0 to 91
Data columns (total 27 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   player               904 non-null    object 
 1   player_id            904 non-null    int64  
 2   position             903 non-null    object 
 3   player_game_count    904 non-null    int64  
 4   team_name            904 non-null    object 
 5   dropbacks            904 non-null    int64  
 6   attempts             904 non-null    int64  
 7   completions          904 non-null    int64  
 8   completion_percent   904 non-null    float64
 9   yards                904 non-null    int64  
 10  ypa                  904 non-null    float64
 11  touchdowns           904 non-null    int64  
 12  interceptions        904 non-null    int64  
 13  grades_offense       904 non-null    float64
 14  grades_pass          904 non-null    float64
 15  grades_run           777 non-null    floa

In [5]:
all_drafts = pd.read_csv('all_drafts.csv')

In [6]:
all_drafts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2718 entries, 0 to 2717
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         2718 non-null   int64  
 1   college_stats_url  2339 non-null   object 
 2   draft_age          2655 non-null   float64
 3   draft_overall      2718 non-null   int64  
 4   draft_round        2718 non-null   int64  
 5   draft_year         2718 non-null   int64  
 6   player             2718 non-null   object 
 7   position           2718 non-null   object 
 8   pro_stats_url      2661 non-null   object 
 9   school             2711 non-null   object 
 10  team_nfl           2718 non-null   object 
dtypes: float64(1), int64(4), object(6)
memory usage: 233.7+ KB


In [7]:
all_drafts.head()

,Unnamed: 0,college_stats_url,draft_age,draft_overall,draft_round,draft_year,player,position,pro_stats_url,school,team_nfl
0,0,http://www.sports-reference.com/cfb/players/ma...,21.0,1,1,2006,Mario Williams,DE,https://www.pro-football-reference.com/players...,North Carolina St.,HOU
1,1,http://www.sports-reference.com/cfb/players/re...,21.0,2,1,2006,Reggie Bush,RB,https://www.pro-football-reference.com/players...,USC,NOR
2,2,http://www.sports-reference.com/cfb/players/vi...,23.0,3,1,2006,Vince Young,QB,https://www.pro-football-reference.com/players...,Texas,TEN
3,3,http://www.sports-reference.com/cfb/players/aj...,22.0,5,1,2006,A.J. Hawk,LB,https://www.pro-football-reference.com/players...,Ohio St.,GNB
4,4,http://www.sports-reference.com/cfb/players/ve...,22.0,6,1,2006,Vernon Davis,TE,https://www.pro-football-reference.com/players...,Maryland,SFO


Below I drop two subsets of data, first I drop data that does not have a url for college stats in my dataframe, as some of the player data I scraped off of pro-football-reference did not have associated college stats records.  Since I cannot use these players in my model I dropped those datapoints.  I also choose to drop players of positions I deemed irrelevant because they had few or no stats associated with their college careers (such as Offensive Linemen, or Punters) on their college stats page, so modeling these positions would be impossible.

In [8]:
all_drafts.dropna(subset=['college_stats_url'], inplace=True)

In [9]:
relevant_pos = ['QB']
irrelevant_pos = [i for i in all_drafts['position'].unique() if i not in relevant_pos]

In [10]:
irrelevant_pos

['DE',
 'RB',
 'LB',
 'TE',
 'DB',
 'DT',
 'WR',
 'FB',
 'NT',
 'DL',
 'OLB',
 'CB',
 'ILB',
 'S']

In [73]:
for i in irrelevant_pos:
    all_drafts.drop(index=all_drafts[all_drafts['position']==i].index, inplace=True)


In [74]:
all_drafts['position'].unique()

array(['QB', nan], dtype=object)

In [13]:
all_drafts.isnull().sum()

Unnamed: 0           0
college_stats_url    0
draft_age            2
draft_overall        0
draft_round          0
draft_year           0
player               0
position             0
pro_stats_url        2
school               0
team_nfl             0
dtype: int64

Below I am looping through the larger all_drafts dataframe and encoding columns for grades, and number of games for each player for their first, second, third, and fourth years in the NFL.

In [14]:
all_drafts['games_y1'] = np.nan
all_drafts['ov_grade_y1'] = np.nan
all_drafts['games_y2'] = np.nan
all_drafts['ov_grade_y2'] = np.nan
all_drafts['games_y3'] = np.nan
all_drafts['ov_grade_y3'] = np.nan
all_drafts['games_y4'] = np.nan
all_drafts['ov_grade_y4'] = np.nan

for index, row in all_drafts.iterrows():
    print(row['player'])

    if row['position'] == 'QB':

        try:
            passing_mask = passing_df['key']== row['player']+str(row['draft_year'])
            passing_index = passing_df[passing_mask].index[0]
            all_drafts.loc[index, 'games_y1'] = passing_df[passing_mask].loc[passing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y1'] = passing_df[passing_mask].loc[passing_index, 'grades_offense']
        except IndexError:
            pass

        try:
            passing_mask = passing_df['key'] == row['player']+str(row['draft_year']+1)
            passing_index = passing_df[passing_mask].index[0]
            all_drafts.loc[index, 'games_y2'] = passing_df[passing_mask].loc[passing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y2'] = passing_df[passing_mask].loc[passing_index, 'grades_offense']
        except IndexError:
            pass

        try:
            passing_mask = passing_df['key'] == row['player']+str(row['draft_year']+2)
            passing_index = passing_df[passing_mask].index[0]
            all_drafts.loc[index, 'games_y3'] = passing_df[passing_mask].loc[passing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y3'] = passing_df[passing_mask].loc[passing_index, 'grades_offense']
        except IndexError:
            pass

        try:
            passing_mask = passing_df['key'] == row['player']+str(row['draft_year']+2)
            passing_index = passing_df[passing_mask].index[0]
            all_drafts.loc[index, 'games_y4'] = passing_df[passing_mask].loc[passing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y4'] = passing_df[passing_mask].loc[passing_index, 'grades_offense']
        except IndexError:
            pass


    elif (row['position'] == 'RB') | (row['position'] == 'FB'):
        try:
            rushing_mask = rushing_df['key']== row['player']+str(row['draft_year'])
            rushing_index = rushing_df[rushing_mask].index[0]
            all_drafts.loc[index, 'games_y1'] = rushing_df[rushing_mask].loc[rushing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y1'] = rushing_df[rushing_mask].loc[rushing_index, 'grades_offense']
        except:
            pass

        try:
            rushing_mask = rushing_df['key']== row['player']+str(row['draft_year']+1)
            rushing_index = rushing_df[rushing_mask].index[0]
            all_drafts.loc[index, 'games_y2'] = rushing_df[rushing_mask].loc[rushing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y2'] = rushing_df[rushing_mask].loc[rushing_index, 'grades_offense']
        except IndexError:
            pass

        try:
            rushing_mask = rushing_df['key']== row['player']+str(row['draft_year']+2)
            rushing_index = rushing_df[rushing_mask].index[0]
            all_drafts.loc[index, 'games_y3'] = rushing_df[rushing_mask].loc[rushing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y3'] = rushing_df[rushing_mask].loc[rushing_index, 'grades_offense']

        except IndexError:
            pass

        try:
            rushing_mask = rushing_df['key']== row['player']+str(row['draft_year']+3)
            rushing_index = rushing_df[rushing_mask].index[0]
            all_drafts.loc[index, 'games_y4'] = rushing_df[rushing_mask].loc[rushing_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y4'] = rushing_df[rushing_mask].loc[rushing_index, 'grades_offense']

        except:
            pass


    elif (row['position'] == 'WR') | (row['position'] == 'TE'):
        try:
            receiving_mask = receiving_df['key']== row['player']+str(row['draft_year'])
            receiving_index = receiving_df[receiving_mask].index[0]
            all_drafts.loc[index, 'games_y1'] = receiving_df[receiving_mask].loc[receiving_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y1'] = receiving_df[receiving_mask].loc[receiving_index, 'grades_offense']
        except IndexError:
            pass

        try:
            receiving_mask = receiving_df['key']== row['player']+str(row['draft_year']+1)
            receiving_index = receiving_df[receiving_mask].index[0]
            all_drafts.loc[index, 'games_y2'] = receiving_df[receiving_mask].loc[receiving_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y2'] = receiving_df[receiving_mask].loc[receiving_index, 'grades_offense']
        except IndexError:
            pass

        try:
            receiving_mask = receiving_df['key']== row['player']+str(row['draft_year']+2)
            receiving_index = receiving_df[receiving_mask].index[0]
            all_drafts.loc[index, 'games_y3'] = receiving_df[receiving_mask].loc[receiving_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y3'] = receiving_df[receiving_mask].loc[receiving_index, 'grades_offense']
        except IndexError:
            pass

        try:
            receiving_mask = receiving_df['key']== row['player']+str(row['draft_year']+3)
            receiving_index = receiving_df[receiving_mask].index[0]
            all_drafts.loc[index, 'games_y4'] = receiving_df[receiving_mask].loc[receiving_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y4'] = receiving_df[receiving_mask].loc[receiving_index, 'grades_offense']

        except IndexError:
            pass

    elif row['position'] in ['CB', 'DE','ILB', 'S', 'DT', 'OLB', 'LB', 'DB', 'DL', 'NT']:
        try:
            defense_mask = defense_df['key']== row['player']+str(row['draft_year'])
            defense_index = defense_df[defense_mask].index[0]
            all_drafts.loc[index, 'games_y1'] = defense_df[defense_mask].loc[defense_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y1'] = defense_df[defense_mask].loc[defense_index, 'grades_defense']
        except IndexError:
            pass

        try:
            defense_mask = defense_df['key']== row['player']+str(row['draft_year']+1)
            defense_index = defense_df[defense_mask].index[0]
            all_drafts.loc[index, 'games_y2'] = defense_df[defense_mask].loc[defense_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y2'] = defense_df[defense_mask].loc[defense_index, 'grades_defense']
        except IndexError:
            pass

        try:
            defense_mask = defense_df['key']== row['player']+str(row['draft_year']+2)
            defense_index = defense_df[defense_mask].index[0]
            all_drafts.loc[index, 'games_y3'] = defense_df[defense_mask].loc[defense_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y3'] = defense_df[defense_mask].loc[defense_index, 'grades_defense']
        except IndexError:
            pass

        try:
            defense_mask = defense_df['key']== row['player']+str(row['draft_year']+3)
            defense_index = defense_df[defense_mask].index[0]
            all_drafts.loc[index, 'games_y4'] = defense_df[defense_mask].loc[defense_index, 'player_game_count']
            all_drafts.loc[index, 'ov_grade_y4'] = defense_df[defense_mask].loc[defense_index, 'grades_defense']
        except IndexError:
            pass


Vince Young
Matt Leinart
Jay Cutler
Kellen Clemens
Tarvaris Jackson
Charlie Whitehurst
Brodie Croyle
Brad Smith
Omar Jacobs
Reggie McNeal
Bruce Gradkowski
D.J. Shockley
JaMarcus Russell
Brady Quinn
Kevin Kolb
John Beck
Drew Stanton
Trent Edwards
Isaiah Stanback
Troy Smith
Jordan Palmer
Matt Ryan
Joe Flacco
Brian Brohm
Chad Henne
Kevin O'Connell
Dennis Dixon
Erik Ainge
Colt Brennan
Andre Woodson
Matt Flynn
Alex Brink
Matthew Stafford
Mark Sanchez
Josh Freeman
Pat White
Stephen McGee
Tom Brandstater
Curtis Painter
Sam Bradford
Tim Tebow
Jimmy Clausen
Colt McCoy
Mike Kafka
Rusty Smith
Tony Pike
Levi Brown
Cam Newton
Jake Locker
Blaine Gabbert
Christian Ponder
Andy Dalton
Colin Kaepernick
Ryan Mallett
T.J. Yates
Nathan Enderle
Tyrod Taylor
Greg McElroy
Andrew Luck
Robert Griffin
Ryan Tannehill
Brandon Weeden
Brock Osweiler
Russell Wilson
Nick Foles
Kirk Cousins
Ryan Lindley
Chandler Harnish
EJ Manuel
Geno Smith
Mike Glennon
Matt Barkley
Ryan Nassib
Tyler Wilson
Landry Jones
Zac Dysert
B.J.

In [15]:
all_drafts.columns

Index(['Unnamed: 0', 'college_stats_url', 'draft_age', 'draft_overall',
       'draft_round', 'draft_year', 'player', 'position', 'pro_stats_url',
       'school', 'team_nfl', 'games_y1', 'ov_grade_y1', 'games_y2',
       'ov_grade_y2', 'games_y3', 'ov_grade_y3', 'games_y4', 'ov_grade_y4'],
      dtype='object')

In [16]:
all_drafts[['player', 'position', 'draft_year', 'games_y1', 'ov_grade_y1','games_y2', 'ov_grade_y2', 'games_y3',
            'ov_grade_y3', 'games_y4', 'ov_grade_y4']]

,player,position,draft_year,games_y1,ov_grade_y1,games_y2,ov_grade_y2,games_y3,ov_grade_y3,games_y4,ov_grade_y4
2,Vince Young,QB,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Matt Leinart,QB,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Jay Cutler,QB,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,Kellen Clemens,QB,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,Tarvaris Jackson,QB,2006,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
2671,Luke Falk,QB,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2675,Tanner Lee,QB,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2687,Danny Etling,QB,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2688,Alex McGough,QB,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


from matplotlib import pyplot as plt
_df_0['draft_year'].plot(kind='hist', bins=20, title='draft_year')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['games_y1'].plot(kind='hist', bins=20, title='games_y1')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['ov_grade_y1'].plot(kind='hist', bins=20, title='ov_grade_y1')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['games_y2'].plot(kind='hist', bins=20, title='games_y2')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['draft_year']
  ys = series['ov_grade_y2']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values('draft_year', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('draft_year')
_ = plt.ylabel('ov_grade_y2')

from matplotlib import pyplot as plt
_df_12['draft_year'].plot(kind='line', figsize=(8, 4), title='draft_year')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['games_y1'].plot(kind='line', figsize=(8, 4), title='games_y1')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['ov_grade_y1'].plot(kind='line', figsize=(8, 4), title='ov_grade_y1')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15['games_y2'].plot(kind='line', figsize=(8, 4), title='games_y2')
plt.gca().spines[['top', 'right']].set_visible(False)

In [17]:
all_drafts.isnull().sum()

Unnamed: 0            0
college_stats_url     0
draft_age             2
draft_overall         0
draft_round           0
draft_year            0
player                0
position              0
pro_stats_url         2
school                0
team_nfl              0
games_y1             74
ov_grade_y1          74
games_y2             72
ov_grade_y2          72
games_y3             75
ov_grade_y3          75
games_y4             75
ov_grade_y4          75
dtype: int64

In [18]:
len(all_drafts)

134

In [19]:
for index, row in all_drafts.head().iterrows():
    print(np.isnan(row['games_y2']))

True
True
True
True
True


I created the below variables to examine how many players in my dataframe did not end up playing any years in the NFL.  I did not drop these however because those would be important datapoints, indicating that these players were not successful compared to someone who did play.

In [20]:
all_drafts['played_y1'] = all_drafts['games_y1'].map(lambda x: 1 if np.isnan(x) == False else 0)

In [21]:
all_drafts['played_y2'] = all_drafts['games_y2'].map(lambda x: 1 if np.isnan(x) == False else 0)

In [22]:
all_drafts['played_y3'] = all_drafts['games_y3'].map(lambda x: 1 if np.isnan(x) == False else 0)

In [23]:
all_drafts['played_y4'] = all_drafts['games_y4'].map(lambda x: 1 if np.isnan(x) == False else 0)

In [24]:
all_drafts['years_played'] = all_drafts.apply(lambda row: row['played_y1'] + row['played_y2'] + row['played_y3'] + row['played_y4'], axis=1)

In [25]:
all_drafts[['player', 'played_y1', 'played_y2', 'played_y3', 'played_y4', 'years_played']]

,player,played_y1,played_y2,played_y3,played_y4,years_played
2,Vince Young,0,0,0,0,0
8,Matt Leinart,0,0,0,0,0
9,Jay Cutler,0,0,0,0,0
42,Kellen Clemens,0,0,0,0,0
52,Tarvaris Jackson,0,0,0,0,0
...,...,...,...,...,...,...
2671,Luke Falk,0,0,0,0,0
2675,Tanner Lee,0,0,0,0,0
2687,Danny Etling,0,0,0,0,0
2688,Alex McGough,0,0,0,0,0


In [26]:
all_drafts['years_played'].value_counts()

0    45
4    32
2    25
1    17
3    15
Name: years_played, dtype: int64

In [29]:
college_stats = pd.read_csv('all_player_stats.csv')

In [30]:
college_stats.head()

,Unnamed: 0,player,year,school,conference,team_year_url,class,pos,games,pass_cmp,...,punt_ret_yds,punt_ret_yds_per_ret,punt_ret_td,kick_ret,kick_ret_yds,kick_ret_yds_per_ret,kick_ret_td,losses,rank,wins
0,0,Mario Williams,2005,North Carolina State,ACC,https://www.sports-reference.com/cfb/schools/n...,JR,DL,12.0,NaN,...,7.0,7.0,0.0,NaN,NaN,NaN,NaN,5.0,37.0,7.0
1,1,Reggie Bush,2003,USC,Pac-10,https://www.sports-reference.com/cfb/schools/s...,FR,RB,13.0,NaN,...,4.0,2.0,0.0,18.0,492.0,27.3,1.0,1.0,4.0,12.0
2,2,Reggie Bush,2004,USC,Pac-10,https://www.sports-reference.com/cfb/schools/s...,SO,RB,13.0,1.0,...,376.0,15.7,2.0,21.0,537.0,25.6,0.0,0.0,1.0,13.0
3,3,Reggie Bush,2005,USC,Pac-10,https://www.sports-reference.com/cfb/schools/s...,JR,RB,13.0,0.0,...,179.0,9.9,1.0,28.0,493.0,17.6,0.0,1.0,2.0,12.0
4,4,Vince Young,2003,Texas,Big 12,https://www.sports-reference.com/cfb/schools/t...,FR,QB,12.0,84.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,13.0,10.0


In [31]:
college_stats.isnull().sum()

Unnamed: 0                 0
player                     0
year                       0
school                     0
conference                 1
team_year_url              5
class                     95
pos                        1
games                    217
pass_cmp                7000
pass_att                7000
pass_cmp_pct            7135
pass_yds                7000
pass_yds_per_att        7135
adj_pass_yds_per_att    7135
pass_td                 7000
pass_int                7000
pass_rating             7135
rush_att                5155
rush_yds                5155
rush_yds_per_att        5662
rush_td                 5155
rec                     4761
rec_yds                 4761
rec_yds_per_rec         5114
rec_td                  4761
scrim_att               4163
scrim_yds               4163
scrim_yds_per_att       4566
scrim_td                4163
tackles_solo            1640
tackles_assists         1640
tackles_total           1640
tackles_loss            1640
sacks         

In [32]:
college_stats[['pass_cmp', 'pass_att', 'pass_cmp_pct', 'pass_yds',
       'pass_yds_per_att', 'adj_pass_yds_per_att', 'pass_td', 'pass_int',
       'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td',
       'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td', 'tackles_solo', 'tackles_assists',
       'tackles_total', 'tackles_loss', 'sacks', 'def_int', 'def_int_yds',
       'def_int_yds_per_int', 'def_int_td', 'pass_defended', 'fumbles_rec',
       'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced', 'punt_ret',
       'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td', 'kick_ret',
       'kick_ret_yds', 'kick_ret_yds_per_ret', 'kick_ret_td']] = college_stats[['pass_cmp', 'pass_att', 'pass_cmp_pct', 'pass_yds',
       'pass_yds_per_att', 'adj_pass_yds_per_att', 'pass_td', 'pass_int',
       'pass_rating', 'rush_att', 'rush_yds', 'rush_yds_per_att', 'rush_td',
       'rec', 'rec_yds', 'rec_yds_per_rec', 'rec_td', 'scrim_att', 'scrim_yds',
       'scrim_yds_per_att', 'scrim_td', 'tackles_solo', 'tackles_assists',
       'tackles_total', 'tackles_loss', 'sacks', 'def_int', 'def_int_yds',
       'def_int_yds_per_int', 'def_int_td', 'pass_defended', 'fumbles_rec',
       'fumbles_rec_yds', 'fumbles_rec_td', 'fumbles_forced', 'punt_ret',
       'punt_ret_yds', 'punt_ret_yds_per_ret', 'punt_ret_td', 'kick_ret',
       'kick_ret_yds', 'kick_ret_yds_per_ret', 'kick_ret_td']].fillna(value=0)

I chose to fill null values in my college stats dataframe with 0 values, because the assumption is that if it was not included on a table on pro-football-reference.com there were not stats accumulated in that category that year for that player.  Rather than having a table full of zeros for multiple categories, that website opted to just have those specific categories missing.  Because of this I have to assume those values are zeros for those players, however this could be a limitation of my modeling later on, and other than manually checking and imputing values I don't really have a way of knowing whether or not a stat is actually missing, or just not accumulated that season meaning it would be a zero.

In [33]:
college_stats.isnull().sum()

Unnamed: 0                0
player                    0
year                      0
school                    0
conference                1
team_year_url             5
class                    95
pos                       1
games                   217
pass_cmp                  0
pass_att                  0
pass_cmp_pct              0
pass_yds                  0
pass_yds_per_att          0
adj_pass_yds_per_att      0
pass_td                   0
pass_int                  0
pass_rating               0
rush_att                  0
rush_yds                  0
rush_yds_per_att          0
rush_td                   0
rec                       0
rec_yds                   0
rec_yds_per_rec           0
rec_td                    0
scrim_att                 0
scrim_yds                 0
scrim_yds_per_att         0
scrim_td                  0
tackles_solo              0
tackles_assists           0
tackles_total             0
tackles_loss              0
sacks                     0
def_int             

I create two dataframes from the larger college stats dataframe that I have, those being avg_college_stats and final_year_college_stats, as these would be my way of creating one single datapoint for each player that I could map to my draft_picks, rather than having multiple lines for different years of college stats.  Later on in modeling I opt to use the avg_college_stats over final_year_college_stats.

In [ ]:
avg_college_stats = college_stats.groupby('player', sort=False).mean()

In [35]:
avg_college_stats.head()

,Unnamed: 0,year,games,pass_cmp,pass_att,pass_cmp_pct,pass_yds,pass_yds_per_att,adj_pass_yds_per_att,pass_td,...,punt_ret_yds,punt_ret_yds_per_ret,punt_ret_td,kick_ret,kick_ret_yds,kick_ret_yds_per_ret,kick_ret_td,losses,rank,wins
player,,,,,,,,,,,,,,,,,,,,,
Mario Williams,0.0,2005.0,12.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,7.000000,7.0,0.0,0.000000,0.000000,0.000000,0.000000,5.000000,37.000000,7.000000
Reggie Bush,2.0,2004.0,13.000000,0.333333,1.000000,33.333333,17.333333,17.333333,24.000000,0.333333,...,186.333333,9.2,1.0,22.333333,507.333333,23.500000,0.333333,0.666667,2.333333,12.333333
Vince Young,5.0,2004.0,12.333333,148.000000,239.333333,61.033333,2013.333333,8.266667,7.566667,14.666667,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,1.333333,6.333333,11.333333
A.J. Hawk,8.5,2003.5,12.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,2.000000,10.250000,10.750000
Vernon Davis,12.0,2004.0,11.666667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,1.000000,22.666667,7.566667,0.000000,5.000000,47.000000,6.666667


In [36]:
max_year = college_stats.groupby('player', sort=False).max()['year']

<ipython-input-36-175d00adc3b7>:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  max_year = college_stats.groupby('player', sort=False).max()['year']


In [38]:
college_stats['final_year'] = np.nan
for name, year in max_year.items():
    print(name)
    for index, row in college_stats.iterrows():
        if row['player'] == name:
            college_stats.loc[index, 'final_year'] = year

Mario Williams
Reggie Bush
Vince Young
A.J. Hawk
Vernon Davis
Michael Huff
Donte Whitner
Ernie Sims
Matt Leinart
Jay Cutler
Haloti Ngata
Kamerion Wimbley
Brodrick Bunkley
Tye Hill
Jason Allen
Chad Greenway
Bobby Carpenter
Antonio Cromartie


KeyboardInterrupt: 

In [40]:
college_stats[['player', 'year', 'final_year']]

,player,year,final_year
0,Mario Williams,2005,2005.0
1,Reggie Bush,2003,2005.0
2,Reggie Bush,2004,2005.0
3,Reggie Bush,2005,2005.0
4,Vince Young,2003,2005.0
...,...,...,...
8019,Austin Proehl,2016,NaN
8020,Austin Proehl,2017,NaN
8021,Trey Quinn,2014,NaN
8022,Trey Quinn,2015,NaN


In [41]:
final_year_college_stats = college_stats[college_stats['year']==college_stats['final_year']]

In [42]:
final_year_college_stats.head()

,Unnamed: 0,player,year,school,conference,team_year_url,class,pos,games,pass_cmp,...,punt_ret_yds_per_ret,punt_ret_td,kick_ret,kick_ret_yds,kick_ret_yds_per_ret,kick_ret_td,losses,rank,wins,final_year
0,0,Mario Williams,2005,North Carolina State,ACC,https://www.sports-reference.com/cfb/schools/n...,JR,DL,12.0,0.0,...,7.0,0.0,0.0,0.0,0.0,0.0,5.0,37.0,7.0,2005.0
3,3,Reggie Bush,2005,USC,Pac-10,https://www.sports-reference.com/cfb/schools/s...,JR,RB,13.0,0.0,...,9.9,1.0,28.0,493.0,17.6,0.0,1.0,2.0,12.0,2005.0
6,6,Vince Young,2005,Texas,Big 12,https://www.sports-reference.com/cfb/schools/t...,JR,QB,13.0,212.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,13.0,2005.0
10,10,A.J. Hawk,2005,Ohio State,Big Ten,https://www.sports-reference.com/cfb/schools/o...,SR,LB,12.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,8.0,10.0,2005.0
13,13,Vernon Davis,2005,Maryland,ACC,https://www.sports-reference.com/cfb/schools/m...,JR,TE,11.0,0.0,...,0.0,0.0,3.0,68.0,22.7,0.0,6.0,65.0,5.0,2005.0


In [43]:
college_stats.drop(columns='Unnamed: 0', inplace=True)

In [44]:
college_stats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8024 entries, 0 to 8023
Data columns (total 55 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   player                8024 non-null   object 
 1   year                  8024 non-null   int64  
 2   school                8024 non-null   object 
 3   conference            8023 non-null   object 
 4   team_year_url         8019 non-null   object 
 5   class                 7929 non-null   object 
 6   pos                   8023 non-null   object 
 7   games                 7807 non-null   float64
 8   pass_cmp              8024 non-null   float64
 9   pass_att              8024 non-null   float64
 10  pass_cmp_pct          8024 non-null   float64
 11  pass_yds              8024 non-null   float64
 12  pass_yds_per_att      8024 non-null   float64
 13  adj_pass_yds_per_att  8024 non-null   float64
 14  pass_td               8024 non-null   float64
 15  pass_int             

In [45]:
new_cols = college_stats.drop(columns=['player', 'school', 'conference', 'class', 'pos', 'games', 'year', 'final_year']).columns
avg_cols = ['avg_' + i for i in new_cols]
final_year_cols = ['final_year_' + i for i in new_cols]

In [46]:
all_drafts['conference'] = np.nan

for i in avg_cols:
    all_drafts[i] = np.nan

for i  in final_year_cols:
    all_drafts[i] = np.nan

Mapping average college stats to draft picks dataframe with new column names

In [47]:
for index, row in all_drafts.iterrows():
    try:
        all_drafts.loc[index, 'avg_pass_cmp'] = avg_college_stats[avg_college_stats.index==row['player']]['pass_cmp'][0]
        all_drafts.loc[index, 'avg_pass_att'] = avg_college_stats[avg_college_stats.index==row['player']]['pass_att'][0]
        all_drafts.loc[index, 'avg_pass_cmp_pct'] = avg_college_stats[avg_college_stats.index==row['player']]['pass_cmp_pct'][0]
        all_drafts.loc[index, 'avg_pass_yds'] = avg_college_stats[avg_college_stats.index==row['player']]['pass_yds'][0]
        all_drafts.loc[index, 'avg_pass_yds_per_att'] = avg_college_stats[avg_college_stats.index==row['player']]['pass_yds_per_att'][0]
        all_drafts.loc[index, 'avg_adj_pass_yds_per_att'] = avg_college_stats[avg_college_stats.index==row['player']]['adj_pass_yds_per_att'][0]
        all_drafts.loc[index, 'avg_pass_td'] = avg_college_stats[avg_college_stats.index==row['player']]['pass_td'][0]
        all_drafts.loc[index, 'avg_pass_int'] = avg_college_stats[avg_college_stats.index==row['player']]['pass_int'][0]
        all_drafts.loc[index, 'avg_pass_rating'] = avg_college_stats[avg_college_stats.index==row['player']]['pass_rating'][0]
        all_drafts.loc[index, 'avg_rush_att'] = avg_college_stats[avg_college_stats.index==row['player']]['rush_att'][0]
        all_drafts.loc[index, 'avg_rush_yds'] = avg_college_stats[avg_college_stats.index==row['player']]['rush_yds'][0]
        all_drafts.loc[index, 'avg_rush_yds_per_att'] = avg_college_stats[avg_college_stats.index==row['player']]['rush_yds_per_att'][0]
        all_drafts.loc[index, 'avg_rush_td'] = avg_college_stats[avg_college_stats.index==row['player']]['rush_td'][0]
        all_drafts.loc[index, 'avg_rec'] = avg_college_stats[avg_college_stats.index==row['player']]['rec'][0]
        all_drafts.loc[index, 'avg_rec_yds'] = avg_college_stats[avg_college_stats.index==row['player']]['rec_yds'][0]
        all_drafts.loc[index, 'avg_rec_yds_per_rec'] = avg_college_stats[avg_college_stats.index==row['player']]['rec_yds_per_rec'][0]
        all_drafts.loc[index, 'avg_rec_td'] = avg_college_stats[avg_college_stats.index==row['player']]['rec_td'][0]
        all_drafts.loc[index, 'avg_scrim_att'] = avg_college_stats[avg_college_stats.index==row['player']]['scrim_att'][0]
        all_drafts.loc[index, 'avg_scrim_yds'] = avg_college_stats[avg_college_stats.index==row['player']]['scrim_yds'][0]
        all_drafts.loc[index, 'avg_scrim_yds_per_att'] = avg_college_stats[avg_college_stats.index==row['player']]['scrim_yds_per_att'][0]
        all_drafts.loc[index, 'avg_scrim_td'] = avg_college_stats[avg_college_stats.index==row['player']]['scrim_td'][0]
        all_drafts.loc[index, 'avg_tackles_solo'] = avg_college_stats[avg_college_stats.index==row['player']]['tackles_solo'][0]
        all_drafts.loc[index, 'avg_tackles_assists'] = avg_college_stats[avg_college_stats.index==row['player']]['tackles_assists'][0]
        all_drafts.loc[index, 'avg_tackles_total'] = avg_college_stats[avg_college_stats.index==row['player']]['tackles_total'][0]
        all_drafts.loc[index, 'avg_tackles_loss'] = avg_college_stats[avg_college_stats.index==row['player']]['tackles_loss'][0]
        all_drafts.loc[index, 'avg_sacks'] = avg_college_stats[avg_college_stats.index==row['player']]['sacks'][0]
        all_drafts.loc[index, 'avg_def_int'] = avg_college_stats[avg_college_stats.index==row['player']]['def_int'][0]
        all_drafts.loc[index, 'avg_def_int_yds'] = avg_college_stats[avg_college_stats.index==row['player']]['def_int_yds'][0]
        all_drafts.loc[index, 'avg_def_int_yds_per_int'] = avg_college_stats[avg_college_stats.index==row['player']]['def_int_yds_per_int'][0]
        all_drafts.loc[index, 'avg_def_int_td'] = avg_college_stats[avg_college_stats.index==row['player']]['def_int_td'][0]
        all_drafts.loc[index, 'avg_pass_defended'] = avg_college_stats[avg_college_stats.index==row['player']]['pass_defended'][0]
        all_drafts.loc[index, 'avg_fumbles_rec'] = avg_college_stats[avg_college_stats.index==row['player']]['fumbles_rec'][0]
        all_drafts.loc[index, 'avg_fumbles_rec_yds'] = avg_college_stats[avg_college_stats.index==row['player']]['fumbles_rec_yds'][0]
        all_drafts.loc[index, 'avg_fumbles_rec_td'] = avg_college_stats[avg_college_stats.index==row['player']]['fumbles_rec_td'][0]
        all_drafts.loc[index, 'avg_fumbles_forced'] = avg_college_stats[avg_college_stats.index==row['player']]['fumbles_forced'][0]
        all_drafts.loc[index, 'avg_punt_ret'] = avg_college_stats[avg_college_stats.index==row['player']]['punt_ret'][0]
        all_drafts.loc[index, 'avg_punt_ret_yds'] = avg_college_stats[avg_college_stats.index==row['player']]['punt_ret_yds'][0]
        all_drafts.loc[index, 'avg_punt_ret_yds_per_ret'] = avg_college_stats[avg_college_stats.index==row['player']]['punt_ret_yds_per_ret'][0]
        all_drafts.loc[index, 'avg_punt_ret_td'] = avg_college_stats[avg_college_stats.index==row['player']]['punt_ret_td'][0]
        all_drafts.loc[index, 'avg_kick_ret'] = avg_college_stats[avg_college_stats.index==row['player']]['kick_ret'][0]
        all_drafts.loc[index, 'avg_kick_ret_yds'] = avg_college_stats[avg_college_stats.index==row['player']]['kick_ret_yds'][0]
        all_drafts.loc[index, 'avg_kick_ret_yds_per_ret'] = avg_college_stats[avg_college_stats.index==row['player']]['kick_ret_yds_per_ret'][0]
        all_drafts.loc[index, 'avg_kick_ret_td'] = avg_college_stats[avg_college_stats.index==row['player']]['kick_ret_td'][0]
        print(index)
        print(row['player'])
    except IndexError:
        pass

2
Vince Young
8
Matt Leinart
9
Jay Cutler
42
Kellen Clemens
52
Tarvaris Jackson
65
Charlie Whitehurst
69
Brodie Croyle
85
Brad Smith
134
Omar Jacobs
158
Reggie McNeal
159
Bruce Gradkowski
177
D.J. Shockley
202
JaMarcus Russell
221
Brady Quinn
232
Kevin Kolb
235
John Beck
237
Drew Stanton
280
Trent Edwards
290
Isaiah Stanback
347
Troy Smith
369
Jordan Palmer
412
Matt Ryan
423
Joe Flacco
457
Brian Brohm
458
Chad Henne
492
Kevin O'Connell
543
Dennis Dixon
548
Erik Ainge
567
Colt Brennan
577
Andre Woodson
587
Matt Flynn
598
Alex Brink
620
Matthew Stafford
623
Mark Sanchez
633
Josh Freeman
656
Pat White
704
Stephen McGee
760
Tom Brandstater
784
Curtis Painter
830
Sam Bradford
848
Tim Tebow
869
Jimmy Clausen
900
Colt McCoy
933
Mike Kafka
976
Rusty Smith
1000
Tony Pike
1003
Levi Brown
1044
Cam Newton
1051
Jake Locker
1052
Blaine Gabbert
1054
Christian Ponder
1070
Andy Dalton
1071
Colin Kaepernick
1104
Ryan Mallett
1171
T.J. Yates
1179
Nathan Enderle
1193
Tyrod Taylor
1216
Greg McElroy
1256
An

Mapping final year college stats to draft picks dataframe with new column names

In [48]:
for index, row in all_drafts.iterrows():
    try:
        print(index)
        print(row['player'])
        player_index = final_year_college_stats[final_year_college_stats['player']==row['player']].index[0]

        all_drafts.loc[index, 'conference'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['conference'][player_index]
        all_drafts.loc[index, 'final_year_pass_cmp'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['pass_cmp'][player_index]
        all_drafts.loc[index, 'final_year_pass_att'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['pass_att'][player_index]
        all_drafts.loc[index, 'final_year_pass_cmp_pct'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['pass_cmp_pct'][player_index]
        all_drafts.loc[index, 'final_year_pass_yds'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['pass_yds'][player_index]
        all_drafts.loc[index, 'final_year_pass_yds_per_att'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['pass_yds_per_att'][player_index]
        all_drafts.loc[index, 'final_year_adj_pass_yds_per_att'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['adj_pass_yds_per_att'][player_index]
        all_drafts.loc[index, 'final_year_pass_td'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['pass_td'][player_index]
        all_drafts.loc[index, 'final_year_pass_int'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['pass_int'][player_index]
        all_drafts.loc[index, 'final_year_pass_rating'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['pass_rating'][player_index]
        all_drafts.loc[index, 'final_year_rush_att'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['rush_att'][player_index]
        all_drafts.loc[index, 'final_year_rush_yds'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['rush_yds'][player_index]
        all_drafts.loc[index, 'final_year_rush_yds_per_att'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['rush_yds_per_att'][player_index]
        all_drafts.loc[index, 'final_year_rush_td'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['rush_td'][player_index]
        all_drafts.loc[index, 'final_year_rec'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['rec'][player_index]
        all_drafts.loc[index, 'final_year_rec_yds'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['rec_yds'][player_index]
        all_drafts.loc[index, 'final_year_rec_yds_per_rec'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['rec_yds_per_rec'][player_index]
        all_drafts.loc[index, 'final_year_rec_td'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['rec_td'][player_index]
        all_drafts.loc[index, 'final_year_scrim_att'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['scrim_att'][player_index]
        all_drafts.loc[index, 'final_year_scrim_yds'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['scrim_yds'][player_index]
        all_drafts.loc[index, 'final_year_scrim_yds_per_att'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['scrim_yds_per_att'][player_index]
        all_drafts.loc[index, 'final_year_scrim_td'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['scrim_td'][player_index]
        all_drafts.loc[index, 'final_year_tackles_solo'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['tackles_solo'][player_index]
        all_drafts.loc[index, 'final_year_tackles_assists'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['tackles_assists'][player_index]
        all_drafts.loc[index, 'final_year_tackles_total'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['tackles_total'][player_index]
        all_drafts.loc[index, 'final_year_tackles_loss'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['tackles_loss'][player_index]
        all_drafts.loc[index, 'final_year_sacks'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['sacks'][player_index]
        all_drafts.loc[index, 'final_year_def_int'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['def_int'][player_index]
        all_drafts.loc[index, 'final_year_def_int_yds'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['def_int_yds'][player_index]
        all_drafts.loc[index, 'final_year_def_int_yds_per_int'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['def_int_yds_per_int'][player_index]
        all_drafts.loc[index, 'final_year_def_int_td'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['def_int_td'][player_index]
        all_drafts.loc[index, 'final_year_pass_defended'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['pass_defended'][player_index]
        all_drafts.loc[index, 'final_year_fumbles_rec'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['fumbles_rec'][player_index]
        all_drafts.loc[index, 'final_year_fumbles_rec_yds'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['fumbles_rec_yds'][player_index]
        all_drafts.loc[index, 'final_year_fumbles_rec_td'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['fumbles_rec_td'][player_index]
        all_drafts.loc[index, 'final_year_fumbles_forced'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['fumbles_forced'][player_index]
        all_drafts.loc[index, 'final_year_punt_ret'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['punt_ret'][player_index]
        all_drafts.loc[index, 'final_year_punt_ret_yds'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['punt_ret_yds'][player_index]
        all_drafts.loc[index, 'final_year_punt_ret_yds_per_ret'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['punt_ret_yds_per_ret'][player_index]
        all_drafts.loc[index, 'final_year_punt_ret_td'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['punt_ret_td'][player_index]
        all_drafts.loc[index, 'final_year_kick_ret'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['kick_ret'][player_index]
        all_drafts.loc[index, 'final_year_kick_ret_yds'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['kick_ret_yds'][player_index]
        all_drafts.loc[index, 'final_year_kick_ret_yds_per_ret'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['kick_ret_yds_per_ret'][player_index]
        all_drafts.loc[index, 'final_year_kick_ret_td'] = final_year_college_stats[final_year_college_stats['player']==row['player']]['kick_ret_td'][player_index]
    except IndexError:
        pass

2
Vince Young
8
Matt Leinart
9
Jay Cutler
42
Kellen Clemens
52
Tarvaris Jackson
65
Charlie Whitehurst
69
Brodie Croyle
85
Brad Smith
134
Omar Jacobs
158
Reggie McNeal
159
Bruce Gradkowski
177
D.J. Shockley
202
JaMarcus Russell
221
Brady Quinn
232
Kevin Kolb
235
John Beck
237
Drew Stanton
280
Trent Edwards
290
Isaiah Stanback
347
Troy Smith
369
Jordan Palmer
412
Matt Ryan
423
Joe Flacco
457
Brian Brohm
458
Chad Henne
492
Kevin O'Connell
543
Dennis Dixon
548
Erik Ainge
567
Colt Brennan
577
Andre Woodson
587
Matt Flynn
598
Alex Brink
620
Matthew Stafford
623
Mark Sanchez
633
Josh Freeman
656
Pat White
704
Stephen McGee
760
Tom Brandstater
784
Curtis Painter
830
Sam Bradford
848
Tim Tebow
869
Jimmy Clausen
900
Colt McCoy
933
Mike Kafka
976
Rusty Smith
1000
Tony Pike
1003
Levi Brown
1044
Cam Newton
1051
Jake Locker
1052
Blaine Gabbert
1054
Christian Ponder
1070
Andy Dalton
1071
Colin Kaepernick
1104
Ryan Mallett
1171
T.J. Yates
1179
Nathan Enderle
1193
Tyrod Taylor
1216
Greg McElroy
1256
An

In [49]:
all_drafts[final_year_cols]

,final_year_team_year_url,final_year_pass_cmp,final_year_pass_att,final_year_pass_cmp_pct,final_year_pass_yds,final_year_pass_yds_per_att,final_year_adj_pass_yds_per_att,final_year_pass_td,final_year_pass_int,final_year_pass_rating,...,final_year_punt_ret_yds,final_year_punt_ret_yds_per_ret,final_year_punt_ret_td,final_year_kick_ret,final_year_kick_ret_yds,final_year_kick_ret_yds_per_ret,final_year_kick_ret_td,final_year_losses,final_year_rank,final_year_wins
2,NaN,212.0,325.0,65.2,3036.0,9.3,9.6,26.0,10.0,163.9,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
8,NaN,283.0,431.0,65.7,3815.0,8.9,9.3,28.0,8.0,157.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
9,NaN,273.0,462.0,59.1,3073.0,6.7,6.7,21.0,9.0,126.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN
42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2675,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2687,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
all_drafts.drop(columns=['Unnamed: 0'], inplace=True)

In [51]:
for i in all_drafts.columns:
    print(i)

college_stats_url
draft_age
draft_overall
draft_round
draft_year
player
position
pro_stats_url
school
team_nfl
games_y1
ov_grade_y1
games_y2
ov_grade_y2
games_y3
ov_grade_y3
games_y4
ov_grade_y4
played_y1
played_y2
played_y3
played_y4
years_played
conference
avg_team_year_url
avg_pass_cmp
avg_pass_att
avg_pass_cmp_pct
avg_pass_yds
avg_pass_yds_per_att
avg_adj_pass_yds_per_att
avg_pass_td
avg_pass_int
avg_pass_rating
avg_rush_att
avg_rush_yds
avg_rush_yds_per_att
avg_rush_td
avg_rec
avg_rec_yds
avg_rec_yds_per_rec
avg_rec_td
avg_scrim_att
avg_scrim_yds
avg_scrim_yds_per_att
avg_scrim_td
avg_tackles_solo
avg_tackles_assists
avg_tackles_total
avg_tackles_loss
avg_sacks
avg_def_int
avg_def_int_yds
avg_def_int_yds_per_int
avg_def_int_td
avg_pass_defended
avg_fumbles_rec
avg_fumbles_rec_yds
avg_fumbles_rec_td
avg_fumbles_forced
avg_punt_ret
avg_punt_ret_yds
avg_punt_ret_yds_per_ret
avg_punt_ret_td
avg_kick_ret
avg_kick_ret_yds
avg_kick_ret_yds_per_ret
avg_kick_ret_td
avg_losses
avg_rank
avg_

In [52]:
all_drafts[all_drafts.columns[0:50]].isnull().sum()

college_stats_url             0
draft_age                     2
draft_overall                 0
draft_round                   0
draft_year                    0
player                        0
position                      0
pro_stats_url                 2
school                        0
team_nfl                      0
games_y1                     74
ov_grade_y1                  74
games_y2                     72
ov_grade_y2                  72
games_y3                     75
ov_grade_y3                  75
games_y4                     75
ov_grade_y4                  75
played_y1                     0
played_y2                     0
played_y3                     0
played_y4                     0
years_played                  0
conference                  131
avg_team_year_url           134
avg_pass_cmp                  3
avg_pass_att                  3
avg_pass_cmp_pct              3
avg_pass_yds                  3
avg_pass_yds_per_att          3
avg_adj_pass_yds_per_att      3
avg_pass

In [53]:
all_drafts[all_drafts.columns[50:]].isnull().sum()

avg_sacks                            3
avg_def_int                          3
avg_def_int_yds                      3
avg_def_int_yds_per_int              3
avg_def_int_td                       3
                                  ... 
final_year_kick_ret_yds_per_ret    131
final_year_kick_ret_td             131
final_year_losses                  134
final_year_rank                    134
final_year_wins                    134
Length: 68, dtype: int64

In [54]:
all_drafts[all_drafts['player'].isnull()==True]

,college_stats_url,draft_age,draft_overall,draft_round,draft_year,player,position,pro_stats_url,school,team_nfl,...,final_year_punt_ret_yds,final_year_punt_ret_yds_per_ret,final_year_punt_ret_td,final_year_kick_ret,final_year_kick_ret_yds,final_year_kick_ret_yds_per_ret,final_year_kick_ret_td,final_year_losses,final_year_rank,final_year_wins


In [55]:
all_drafts.dropna(subset=['player'], inplace=True)

In [56]:
all_drafts.isnull().sum()

college_stats_url                    0
draft_age                            2
draft_overall                        0
draft_round                          0
draft_year                           0
                                  ... 
final_year_kick_ret_yds_per_ret    131
final_year_kick_ret_td             131
final_year_losses                  134
final_year_rank                    134
final_year_wins                    134
Length: 118, dtype: int64

In [57]:
all_drafts[all_drafts['avg_pass_cmp'].isnull()==True].isnull().sum()

college_stats_url                  0
draft_age                          0
draft_overall                      0
draft_round                        0
draft_year                         0
                                  ..
final_year_kick_ret_yds_per_ret    3
final_year_kick_ret_td             3
final_year_losses                  3
final_year_rank                    3
final_year_wins                    3
Length: 118, dtype: int64

In [58]:
all_drafts[all_drafts['avg_pass_cmp'].isnull()==True]

,college_stats_url,draft_age,draft_overall,draft_round,draft_year,player,position,pro_stats_url,school,team_nfl,...,final_year_punt_ret_yds,final_year_punt_ret_yds_per_ret,final_year_punt_ret_td,final_year_kick_ret,final_year_kick_ret_yds,final_year_kick_ret_yds_per_ret,final_year_kick_ret_td,final_year_losses,final_year_rank,final_year_wins
1257,http://www.sports-reference.com/cfb/players/ro...,22.0,2,1,2012,Robert Griffin,QB,https://www.pro-football-reference.com/players...,Baylor,WAS,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1468,http://www.sports-reference.com/cfb/players/ej...,23.0,16,1,2013,EJ Manuel,QB,https://www.pro-football-reference.com/players...,Florida St.,BUF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2292,http://www.sports-reference.com/cfb/players/mi...,23.0,2,1,2017,Mitchell Trubisky,QB,https://www.pro-football-reference.com/players...,North Carolina,CHI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
college_stats[college_stats['player']=='Mitchell Trubisky']

,player,year,school,conference,team_year_url,class,pos,games,pass_cmp,pass_att,...,punt_ret_yds_per_ret,punt_ret_td,kick_ret,kick_ret_yds,kick_ret_yds_per_ret,kick_ret_td,losses,rank,wins,final_year


#### Any rows that are showing null values for their college statistics at this point are rows that were included in the original college draft table, but for some reason threw an error when their college stats were scraped so they did not get any values.  For the time being I am going to drop these, but it would be good to revisit these in the future and try to include them for more inclusive data.

In [60]:
all_drafts.dropna(subset=['avg_pass_cmp'], inplace=True)

In [61]:
all_drafts.isnull().sum()

college_stats_url                    0
draft_age                            2
draft_overall                        0
draft_round                          0
draft_year                           0
                                  ... 
final_year_kick_ret_yds_per_ret    128
final_year_kick_ret_td             128
final_year_losses                  131
final_year_rank                    131
final_year_wins                    131
Length: 118, dtype: int64

In [62]:
all_drafts[all_drafts['conference'].isnull()==True]

,college_stats_url,draft_age,draft_overall,draft_round,draft_year,player,position,pro_stats_url,school,team_nfl,...,final_year_punt_ret_yds,final_year_punt_ret_yds_per_ret,final_year_punt_ret_td,final_year_kick_ret,final_year_kick_ret_yds,final_year_kick_ret_yds_per_ret,final_year_kick_ret_td,final_year_losses,final_year_rank,final_year_wins
42,http://www.sports-reference.com/cfb/players/ke...,23.0,49,2,2006,Kellen Clemens,QB,https://www.pro-football-reference.com/players...,Oregon,NYJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52,http://www.sports-reference.com/cfb/players/ta...,23.0,64,2,2006,Tarvaris Jackson,QB,https://www.pro-football-reference.com/players...,Alabama St.,MIN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
65,http://www.sports-reference.com/cfb/players/ch...,24.0,81,3,2006,Charlie Whitehurst,QB,https://www.pro-football-reference.com/players...,Clemson,SDG,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,http://www.sports-reference.com/cfb/players/br...,23.0,85,3,2006,Brodie Croyle,QB,https://www.pro-football-reference.com/players...,Alabama,KAN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85,http://www.sports-reference.com/cfb/players/br...,22.0,103,4,2006,Brad Smith,QB,https://www.pro-football-reference.com/players...,Missouri,NYJ,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,http://www.sports-reference.com/cfb/players/lu...,23.0,199,6,2018,Luke Falk,QB,https://www.pro-football-reference.com/players...,Washington St.,TEN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2675,http://www.sports-reference.com/cfb/players/ta...,23.0,203,6,2018,Tanner Lee,QB,https://www.pro-football-reference.com/players...,Nebraska,JAX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2687,http://www.sports-reference.com/cfb/players/da...,24.0,219,7,2018,Danny Etling,QB,https://www.pro-football-reference.com/players...,LSU,NWE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2688,http://www.sports-reference.com/cfb/players/al...,22.0,220,7,2018,Alex McGough,QB,https://www.pro-football-reference.com/players...,Florida International,SEA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Ohio State is in the Big Ten conference, probably the only manual impuation I will do.  

#### The remaining null values for my target variables (games_y1, ov_grade_y1... etc.) that come from ProFootballFocus.com are null if there is no data for that player in the given year meaning they did not play a snap that season.  Since I am attempting to model a players immediate effecivness in the NFL, I am hoping to be able to factor out situations where a player did not play rather than give them a grade of zero for that year, considering factors such as getting injured before the season started, or being second string to an established starter or superstar on a given team.  I will however consider players who did not player at all for their first four years in the NFL a failure, as regular rookie contracts are generally 3 or 4 years long.  

In [63]:
all_drafts['conference'].unique()

array(['Big 12', 'Pac-10', 'SEC', nan], dtype=object)

In [64]:
all_drafts.loc[555, 'conference'] = 'Big Ten'

In [65]:
all_drafts.loc[[555]]

,college_stats_url,draft_age,draft_overall,draft_round,draft_year,player,position,pro_stats_url,school,team_nfl,...,final_year_punt_ret_yds,final_year_punt_ret_yds_per_ret,final_year_punt_ret_td,final_year_kick_ret,final_year_kick_ret_yds,final_year_kick_ret_yds_per_ret,final_year_kick_ret_td,final_year_losses,final_year_rank,final_year_wins
555,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Below I am creating one average grade as an average of a players first four years grades in the NFL.  I created logic such that if a player didn't play in a given year ('games_y1 would be null for example) or they played less than 5 games that year, then that year would not be included in the average.

In [66]:
for index, row in all_drafts.iterrows():
    numerator = 0
    denominator = 0

    if row['games_y1'] > 4:
        numerator += row['ov_grade_y1']
        denominator += 1

    if row['games_y2'] > 4:
        numerator += row['ov_grade_y2']
        denominator += 1

    if row['games_y3'] > 4:
        numerator += row['ov_grade_y3']
        denominator += 1

    if row['games_y4'] > 4:
        numerator += row['ov_grade_y4']
        denominator += 1

    try:
        all_drafts.loc[index, 'avg_grade'] = numerator / denominator
    except ZeroDivisionError:
        all_drafts.loc[index, 'avg_grade'] = 0

In [67]:
all_drafts.isnull().sum()

college_stats_url           1
draft_age                   3
draft_overall               1
draft_round                 1
draft_year                  1
                         ... 
final_year_kick_ret_td    129
final_year_losses         132
final_year_rank           132
final_year_wins           132
avg_grade                   0
Length: 119, dtype: int64

In [68]:
all_drafts.drop(columns=['college_stats_url', 'draft_age', 'pro_stats_url', 'games_y1', 'ov_grade_y1', 'games_y2',
                         'ov_grade_y2', 'games_y3', 'ov_grade_y3', 'games_y4', 'ov_grade_y4', 'played_y1', 'played_y2',
                         'played_y3', 'played_y4', 'years_played'], axis=1).columns

Index(['draft_overall', 'draft_round', 'draft_year', 'player', 'position',
       'school', 'team_nfl', 'conference', 'avg_team_year_url', 'avg_pass_cmp',
       ...
       'final_year_punt_ret_yds_per_ret', 'final_year_punt_ret_td',
       'final_year_kick_ret', 'final_year_kick_ret_yds',
       'final_year_kick_ret_yds_per_ret', 'final_year_kick_ret_td',
       'final_year_losses', 'final_year_rank', 'final_year_wins', 'avg_grade'],
      dtype='object', length=103)

In [69]:
modeling_df = all_drafts[['draft_overall', 'draft_round', 'draft_year', 'player', 'position',
       'school', 'conference', 'team_nfl', 'avg_grade', 'avg_pass_cmp', 'avg_pass_att',
       'avg_pass_cmp_pct', 'avg_pass_yds', 'avg_pass_yds_per_att',
       'avg_adj_pass_yds_per_att', 'avg_pass_td', 'avg_pass_int',
       'avg_pass_rating', 'avg_rush_att', 'avg_rush_yds',
       'avg_rush_yds_per_att', 'avg_rush_td', 'avg_rec', 'avg_rec_yds',
       'avg_rec_yds_per_rec', 'avg_rec_td', 'avg_scrim_att', 'avg_scrim_yds',
       'avg_scrim_yds_per_att', 'avg_scrim_td', 'avg_tackles_solo',
       'avg_tackles_assists', 'avg_tackles_total', 'avg_tackles_loss',
       'avg_sacks', 'avg_def_int', 'avg_def_int_yds',
       'avg_def_int_yds_per_int', 'avg_def_int_td', 'avg_pass_defended',
       'avg_fumbles_rec', 'avg_fumbles_rec_yds', 'avg_fumbles_rec_td',
       'avg_fumbles_forced', 'avg_punt_ret', 'avg_punt_ret_yds',
       'avg_punt_ret_yds_per_ret', 'avg_punt_ret_td', 'avg_kick_ret',
       'avg_kick_ret_yds', 'avg_kick_ret_yds_per_ret', 'avg_kick_ret_td',
       'final_year_pass_cmp', 'final_year_pass_att', 'final_year_pass_cmp_pct',
       'final_year_pass_yds', 'final_year_pass_yds_per_att',
       'final_year_adj_pass_yds_per_att', 'final_year_pass_td',
       'final_year_pass_int', 'final_year_pass_rating', 'final_year_rush_att',
       'final_year_rush_yds', 'final_year_rush_yds_per_att',
       'final_year_rush_td', 'final_year_rec', 'final_year_rec_yds',
       'final_year_rec_yds_per_rec', 'final_year_rec_td',
       'final_year_scrim_att', 'final_year_scrim_yds',
       'final_year_scrim_yds_per_att', 'final_year_scrim_td',
       'final_year_tackles_solo', 'final_year_tackles_assists',
       'final_year_tackles_total', 'final_year_tackles_loss',
       'final_year_sacks', 'final_year_def_int', 'final_year_def_int_yds',
       'final_year_def_int_yds_per_int', 'final_year_def_int_td',
       'final_year_pass_defended', 'final_year_fumbles_rec',
       'final_year_fumbles_rec_yds', 'final_year_fumbles_rec_td',
       'final_year_fumbles_forced', 'final_year_punt_ret',
       'final_year_punt_ret_yds', 'final_year_punt_ret_yds_per_ret',
       'final_year_punt_ret_td', 'final_year_kick_ret',
       'final_year_kick_ret_yds', 'final_year_kick_ret_yds_per_ret',
       'final_year_kick_ret_td']]

In [71]:
modeling_df.to_csv('modeling_data.csv')

Reordered and dropped unneeded columns above and then saved the dataframe that is read for analysis and modeling.